In [2]:
import os
import sys

root = os.path.abspath(os.path.join('..', '..'))
test_dir = os.path.join(root, 'backend', 'docking', 'tests')

In [4]:
# We can use rdkit to generate small molecule conformers necessary
# for Rosetta ligand docking:

from rdkit import Chem
from rdkit.Chem import AllChem

# Take example output from this directory:
example_hits = '2gs6_picks.sdf'
max_confs = 10

supplier = Chem.SDMolSupplier(example_hits)

for i, mol in enumerate(supplier):
    if mol is None:
        continue
    
    # Dump each hit ligand as a separate sdf (required for Rosetta's molfile_to_params.py script):
    lig_name = f'2gs6_hit_{i+1}'
    mol_hit_file = os.path.join(test_dir, f'{lig_name}.sdf')
    Chem.MolToMolFile(mol, mol_hit_file)

    # Rosetta requires all ligand conformers be in a single sdf file
    # for params generation:
    mol_conf_file = os.path.join(test_dir, f'{lig_name}_conformers.sdf')
    conf_ids = AllChem.EmbedMultipleConfs(mol, numConfs=max_confs,
                                          useExpTorsionAnglePrefs=True,
                                          useBasicKnowledge=True)
    with Chem.SDWriter(mol_conf_file) as writer:
        for conf_id in conf_ids:
            writer.write(mol, confId=conf_id)
